In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
%matplotlib inline
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('../input/digit-recognizer/train.csv')

In [ ]:
train

In [ ]:
test=pd.read_csv('../input/digit-recognizer/test.csv')

**Data representation****

In [ ]:

##required for image reprresention else usually you get a Nonetype error
X = (train.iloc[:,1:].values).astype('float32')
y = train.iloc[:,0].values.astype('int32')

In [ ]:
test1=test.copy()
test1 = test1.values.astype('float32')

In [ ]:
#to make the values to be represented neatly and reuired to work with keras API
X = X.reshape(X.shape[0], 28, 28,1)
test1 = test1.reshape(test1.shape[0], 28, 28,1)


In [ ]:
for i in range(10, 13):
    plt.subplot(330 + (i+1))#gives the dimensions of the plot area
    plt.imshow(X[i], cmap=plt.get_cmap('gray'))#considering gray area
    plt.title(y[i])

In [ ]:
plt.imshow(X[10], cmap=plt.get_cmap('gray'))

In [ ]:
X1=train.drop(['label'],axis=1)
y1=train['label']

In [ ]:
X1

In [ ]:
y1

In [ ]:
##to apply keras we normalize the data by diving by 255 as rgb values range from 0-255
X1=X1/255



In [ ]:
X1.head

In [ ]:
y1

In [ ]:
from keras.utils import to_categorical
y1=to_categorical(y1,10)#considering it to be categorised into 10 diff as numbers from 0-9
y1


In [ ]:
#modelling:

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental import preprocessing
X_train,X_test,y_train,y_test=train_test_split(X1,y1,test_size=0.3,random_state=50)

In [ ]:
X1.shape

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
model=keras.Sequential([
    layers.Dense(64,activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(256,activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(512,activation='relu'),
    layers.Dense(10,activation='softmax')
    
    
])

In [ ]:
## we use sparse as multiclass classification and metrics is taken by the model-
##the one which is best suited
model.compile(
optimizer='adam',
loss='categorical_crossentropy',
metrics=['accuracy'],
)

In [ ]:
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=128,
    epochs=200,
    callbacks=[early_stopping],
)

In [ ]:
score=model.evaluate(X_test,y_test)
score[1]

In [ ]:
test=pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
test.shape

In [ ]:
predictions = model.predict(test)

print(predictions)

#submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions_final)+1)),
                         #"Label": predictions_final})
#submissions.to_csv("submission.csv", index=False, header=True)
#print("Finished uploading submission file to csv")

In [ ]:
Label=np.argmax(predictions,axis=1)
ImageId=np.arange(1,28001)
subs=pd.DataFrame(data={"ImageId":ImageId,"Label":Label})

In [ ]:
subs

In [ ]:
subs.to_csv('sub1.csv', index=False)